In [ ]:
!pip install qiskit
!pip install matplotlib

     |████████████████████████████████| 6.1 MB 5.3 MB/s 
     |████████████████████████████████| 17.9 MB 190 kB/s 
     |████████████████████████████████| 236 kB 43.0 MB/s 
     |████████████████████████████████| 207 kB 34.4 MB/s 
     |████████████████████████████████| 2.1 MB 49.7 MB/s 
     |████████████████████████████████| 634 kB 48.8 MB/s 
     |████████████████████████████████| 1.4 MB 45.1 MB/s 
     |████████████████████████████████| 52 kB 1.2 MB/s 
     |████████████████████████████████| 49 kB 4.3 MB/s 
     |████████████████████████████████| 943 kB 42.1 MB/s 
     |████████████████████████████████| 38.2 MB 1.1 MB/s 
     |████████████████████████████████| 3.5 MB 54.6 MB/s 
     |████████████████████████████████| 6.3 MB 54.7 MB/s 
  Created wheel for qiskit: filename=qiskit-0.31.0-py3-none-any.whl size=11728 sha256=1576efac85065fb5aa893d9bfe3c892e5797e2b5266879e06b9dcc3ba1d9f803
  Stored in directory: /root/.cache/pip/wheels/29/dd/b2/1fe1a9ac92aaf75b267d893ae27329ea229f292a2930

In [ ]:
import numpy as np
import math
import qiskit
from qiskit import circuit
from qiskit.circuit.random import random_circuit

import matplotlib as mpl
from qiskit.quantum_info import PTM, Chi, Statevector, DensityMatrix, partial_trace
from qiskit import transpile, QuantumCircuit, QuantumRegister
import qiskit.quantum_info as qi
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel, amplitude_damping_error
from qiskit.tools.visualization import plot_histogram

#I am here
print(np.matmul(np.array([[0, 1], [1, 0]]), np.array([[0.5, 0.5], [0.5, 0.5]])))

def state_tomography(p):
    I = np.trace(np.matmul(np.array([[1, 0],[0, 1]]), np.array(p)))
    X = np.trace(np.matmul(np.array([[0, 1],[1, 0]]), np.array(p)))
    Y = np.trace(np.matmul(np.array([[0, 0- 1j],[1j, 0]]), np.array(p)))
    Z = np.trace(np.matmul(np.array([[1, 0],[0, -1]]), np.array(p)))
    return np.array([I, X, Y, Z])

def main():

    def PTM():
        #0
        circ0 = QuantumCircuit(2)
        circ0.initialize([1, 0], 0)
        circ0.initialize([1, 0], 1)

        #Replace this with random gates (same random gates for each initialization)
        #We need to store the specific series of operations somewhere
        '''
        below is a place-holder for random rx gates, ry gates etc

        write code to generate a random circuit

        
        '''
        circ0.rx(math.pi/5, 0)
        circ0.ry(math.pi/7, 0)
        circ0.rz(math.pi/9, 0)

        state_in0 = Statevector.from_int(0, 2**2)
        rho_in0 = DensityMatrix(state_in0)
        partial_rho_in0 = partial_trace(rho_in0, [1])
        state_out0 = state_in0.evolve(circ0)
        rho_out0 = DensityMatrix(state_out0)
        partial_rho_out0 = partial_trace(rho_out0, [1])

        #1
        circ1 = QuantumCircuit(2)
        circ1.initialize([0, 1], 0)
        circ1.initialize([1, 0], 1)

        #Replace this with random gates (same random gates for each initialization)
        circ1.rx(math.pi/5, 0)
        circ1.ry(math.pi/7, 0)
        circ1.rz(math.pi/9, 0)

        state_in1 = Statevector.from_int(0, 2**2)
        rho_in1 = DensityMatrix(state_in1)
        partial_rho_in1 = partial_trace(rho_in1, [1])
        state_out1 = state_in0.evolve(circ1)
        rho_out1 = DensityMatrix(state_out1)
        partial_rho_out1 = partial_trace(rho_out1, [1])

        #+
        circX = QuantumCircuit(2)
        circX.initialize([1/math.sqrt(2), 1/math.sqrt(2)], 0)
        circX.initialize([1, 0], 1)

        #Replace this with random gates (same random gates for each initialization)
        circX.rx(math.pi/5, 0)
        circX.ry(math.pi/7, 0)
        circX.rz(math.pi/9, 0)

        state_inX = Statevector.from_int(0, 2**2)
        rho_inX = DensityMatrix(state_inX)
        partial_rho_inX = partial_trace(rho_inX, [1])
        state_outX = state_in0.evolve(circX)
        rho_outX = DensityMatrix(state_outX)
        partial_rho_outX = partial_trace(rho_outX, [1])

        print(partial_rho_outX)
        print((np.array([[0, 1],[1, 0]])* np.array(partial_rho_outX)).trace())

        #i
        circY = QuantumCircuit(2)
        circY.initialize([1/math.sqrt(2), 1j/math.sqrt(2)], 0)
        circY.initialize([1, 0], 1)

        #Replace this with random gates (same random gates for each initialization)
        circY.rx(math.pi/5, 0)
        circY.ry(math.pi/7, 0)
        circY.rz(math.pi/9, 0)

        state_inY = Statevector.from_int(0, 2**2)
        rho_inY = DensityMatrix(state_inY)
        partial_rho_inY = partial_trace(rho_inY, [1])
        state_outY = state_in0.evolve(circY)
        rho_outY = DensityMatrix(state_outY)
        partial_rho_outY = partial_trace(rho_outY, [1])

        #systems of equations next this gives PTM.

        #state_tomography(partial_rho_in0) = state_tomography(partial_rho_out0)
        #state_tomography(partial_rho_in1) = state_tomography(partial_rho_out1)
        #state_tomography(partial_rho_inX) = state_tomography(partial_rho_outX)
        #state_tomography(partial_rho_inY) = state_tomography(partial_rho_outY)

        print(state_tomography(partial_rho_in0), "=", state_tomography(partial_rho_out0))
        print(state_tomography(partial_rho_in1), "=", state_tomography(partial_rho_out1))
        print(state_tomography(partial_rho_inX), '=', state_tomography(partial_rho_outX))
        print(state_tomography(partial_rho_inY), '=', state_tomography(partial_rho_outY))

        #The above isn't code, but those will be the equations

        # I = np.array([[1, 0],[0, 1]])
        # X = np.array([[0, 1],[1, 0]])
        # Y = np.array([[0, 0- 1j],[1j, 0]])
        # Z = np.array([[1, 0],[0, -1]])

        # reference_list = [I, X, Y, Z]

        # R = [ [ 0 for i in range(4) ] for j in range(4) ]
        # for i in range(4):
        #     for j in range(4):
        #         R[i][j] = 0.5*np.real((solutions_list[j]*reference_list[i]).trace)
        
        # return R
    
    PTM()


    def noisy_PTM():

        noiseAD = math.pi/9
        noiseDephasing = math.pi/10

        #0
        circ0 = QuantumCircuit(2)
        circ0.initialize([1, 0], 0)
        circ0.initialize([1, 0], 1)

        #This time the random gates have noise (same gates)
        circ0.rx(math.pi/5, 0)

        #Amplitude Damping
        circ0.cry(noiseAD, 0, 1)
        circ0.cnot(1, 0)
        circ0.reset(1)

        #Dephasing
        circ0.ry(noiseDephasing, 1)
        circ0.cz(1, 0)
        circ0.reset(1)

        circ0.ry(math.pi/7, 0)

        circ0.cry(noiseAD, 0, 1)
        circ0.cnot(1, 0)
        circ0.reset(1)

        circ0.ry(noiseDephasing, 1)
        circ0.cz(1, 0)
        circ0.reset(1)

        circ0.rz(math.pi/9, 0)

        circ0.cry(noiseAD, 0, 1)
        circ0.cnot(1, 0)
        circ0.reset(1)

        circ0.ry(noiseDephasing, 1)
        circ0.cz(1, 0)
        circ0.reset(1)

        state_in0 = Statevector.from_int(0, 2**2)
        rho_in0 = DensityMatrix(state_in0)
        partial_rho_in0 = partial_trace(rho_in0, [1])
        state_out0 = state_in0.evolve(circ0)
        rho_out0 = DensityMatrix(state_out0)
        partial_rho_out0 = partial_trace(rho_out0, [1])

        #1
        circ1 = QuantumCircuit(2)
        circ1.initialize([0, 1], 0)
        circ1.initialize([1, 0], 1)

        #This time the random gates have noise (same gates)
        circ1.rx(math.pi/5, 0)

        #Amplitude Damping
        circ1.cry(noiseAD, 0, 1)
        circ1.cnot(1, 0)
        circ1.reset(1)

        #Dephasing
        circ1.ry(noiseDephasing, 1)
        circ1.cz(1, 0)
        circ1.reset(1)

        circ1.ry(math.pi/7, 0)

        circ1.cry(noiseAD, 0, 1)
        circ1.cnot(1, 0)
        circ1.reset(1)

        circ1.ry(noiseDephasing, 1)
        circ1.cz(1, 0)
        circ1.reset(1)

        circ1.rz(math.pi/9, 0)

        circ1.cry(noiseAD, 0, 1)
        circ1.cnot(1, 0)
        circ1.reset(1)

        circ1.ry(noiseDephasing, 1)
        circ1.cz(1, 0)
        circ1.reset(1)

        state_in1 = Statevector.from_int(0, 2**2)
        rho_in1 = DensityMatrix(state_in1)
        partial_rho_in1 = partial_trace(rho_in1, [1])
        state_out1 = state_in0.evolve(circ1)
        rho_out1 = DensityMatrix(state_out1)
        partial_rho_out1 = partial_trace(rho_out1, [1])

        #+
        circX = QuantumCircuit(2)
        circX.initialize([1/math.sqrt(2), 1/math.sqrt(2)], 0)
        circX.initialize([1, 0], 1)

        #This time the random gates have noise (same gates)
        circX.rx(math.pi/5, 0)

        #Amplitude Damping
        circX.cry(noiseAD, 0, 1)
        circX.cnot(1, 0)
        circX.reset(1)

        #Dephasing
        circX.ry(noiseDephasing, 1)
        circX.cz(1, 0)
        circX.reset(1)

        circX.ry(math.pi/7, 0)

        circX.cry(noiseAD, 0, 1)
        circX.cnot(1, 0)
        circX.reset(1)

        circX.ry(noiseDephasing, 1)
        circX.cz(1, 0)
        circX.reset(1)

        circX.rz(math.pi/9, 0)

        circX.cry(noiseAD, 0, 1)
        circX.cnot(1, 0)
        circX.reset(1)

        circX.ry(noiseDephasing, 1)
        circX.cz(1, 0)
        circX.reset(1)

        state_inX = Statevector.from_int(0, 2**2)
        rho_inX = DensityMatrix(state_inX)
        partial_rho_inX = partial_trace(rho_inX, [1])
        state_outX = state_in0.evolve(circX)
        rho_outX = DensityMatrix(state_outX)
        partial_rho_outX = partial_trace(rho_outX, [1])

        #i
        circY = QuantumCircuit(2)
        circY.initialize([1/math.sqrt(2), 1j/math.sqrt(2)], 0)
        circY.initialize([1, 0], 1)

        #This time the random gates have noise (same gates)
        circY.rx(math.pi/5, 0)

        #Amplitude Damping
        circY.cry(noiseAD, 0, 1)
        circY.cnot(1, 0)
        circY.reset(1)

        #Dephasing
        circY.ry(noiseDephasing, 1)
        circY.cz(1, 0)
        circY.reset(1)

        circY.ry(math.pi/7, 0)

        circY.cry(noiseAD, 0, 1)
        circY.cnot(1, 0)
        circY.reset(1)

        circY.ry(noiseDephasing, 1)
        circY.cz(1, 0)
        circY.reset(1)

        circY.rz(math.pi/9, 0)

        circY.cry(noiseAD, 0, 1)
        circY.cnot(1, 0)
        circY.reset(1)

        circY.ry(noiseDephasing, 1)
        circY.cz(1, 0)
        circY.reset(1)

        state_inY = Statevector.from_int(0, 2**2)
        rho_inY = DensityMatrix(state_inY)
        partial_rho_inY = partial_trace(rho_inY, [1])
        state_outY = state_in0.evolve(circY)
        rho_outY = DensityMatrix(state_outY)
        partial_rho_outY = partial_trace(rho_outY, [1])

        #systems of equations next this gives PTM.

main()
 

[[0.5 0.5]
 [0.5 0.5]]
DensityMatrix([[0.28305813+0.j        , 0.4233169 -0.15407475j],
               [0.4233169 +0.15407475j, 0.71694187+0.j        ]],
              dims=(2,))
0j
[1.+0.j 0.+0.j 0.+0.j 1.+0.j] = [ 1.        +0.j  0.53088466+0.j -0.43228179+0.j  0.72889913+0.j]
[1.+0.j 0.+0.j 0.+0.j 1.+0.j] = [ 1.        +0.j -0.53088466+0.j  0.43228179+0.j -0.72889913+0.j]
[1.+0.j 0.+0.j 0.+0.j 1.+0.j] = [ 1.        +0.j  0.8466338 +0.j  0.3081495 +0.j -0.43388374+0.j]
[1.+0.j 0.+0.j 0.+0.j 1.+0.j] = [ 1.        +0.j -0.03704986+0.j  0.84745286+0.j  0.52957621+0.j]


In [35]:
import random

# Function returns a list of 4 random distributions as a list - [i_dist, x_dist, y_dist, z_dist]
def calc_randomized_distributions():
  r = [random.random() for i in range(4)]
  r_sum = sum(r)
  r = [(i/r_sum) for i in r]
  return r

# Testing
r = calc_randomized_distributions()
print(r)
print(sum(r))

[0.37432394915397604, 0.11876045972530375, 0.44870078733389135, 0.0582148037868289]
1.0
